# Ekstraksi Informasi dari Berita Kriminal Indonesia menggunakan Named Entity Recognition (NER).

## Setting Up
### Install the required libraries

In [14]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory as SastrawiStopWordRemoverFactory
nltk.download('punkt')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Dataset

In [2]:
df = pd.read_csv("../data/processed/cleaned_all_data.csv", sep=',')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907


## Preprocessing
### Tokenization

In [ ]:
def remove_punctuation(text):
    if not isinstance(text, str):
        text = str(text)
    return re.sub(r'[^\w\s]', '', text)

df['judul_processed'] = df['judul'].apply(remove_punctuation)

df['isi_berita_processed'] = df['isi_berita'].apply(remove_punctuation)

print("Hasil setelah membersihkan tanda baca secara terpisah:")
print(df[['judul', 'judul_processed', 'isi_berita', 'isi_berita_processed']].head())

Hasil setelah membersihkan tanda baca secara terpisah:
                                               judul  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...   
2  Pekerja Kafe Disiram Air Keras hingga Wajah 'B...   
3  Hilang Nyawa Pria di Simalungun gegara Rebutan...   
4  Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...   

                                     judul_processed  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...   
2  Pekerja Kafe Disiram Air Keras hingga Wajah Be...   
3  Hilang Nyawa Pria di Simalungun gegara Rebutan...   
4  Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...   

                                          isi_berita  \
0  Pelaku pembunuhan dan pemerkosaan AA (14) seor...   
1  Dalam waktu kurang lebih 3 bulan, Polres Probo...   
2  Seorang pekerja kafe di Cengkareng, MAS (32), ...   
3  Hanya gegara rebutan mikrofon untuk bernyany

In [9]:
df['judul'] = df['judul'].fillna('')
df['isi_berita'] = df['isi_berita'].fillna('')

# Tokenisasi 
df['judul_tokens'] = df['judul'].apply(word_tokenize)
df['isi_berita_tokens'] = df['isi_berita'].apply(word_tokenize)

# Hasil tokenisasi
df[['judul', 'judul_tokens', 'isi_berita', 'isi_berita_tokens']].head()

,judul,judul_tokens,isi_berita,isi_berita_tokens
0,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,"[4, Anak, di, Bawah, Umur, Tersangka, Pembunuh...",Pelaku pembunuhan dan pemerkosaan AA (14) seor...,"[Pelaku, pembunuhan, dan, pemerkosaan, AA, (, ..."
1,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,"[26, Tersangka, Dibekuk, Selama, 3, Bulan, Ter...","Dalam waktu kurang lebih 3 bulan, Polres Probo...","[Dalam, waktu, kurang, lebih, 3, bulan, ,, Pol..."
2,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,"[Pekerja, Kafe, Disiram, Air, Keras, hingga, W...","Seorang pekerja kafe di Cengkareng, MAS (32), ...","[Seorang, pekerja, kafe, di, Cengkareng, ,, MA..."
3,Hilang Nyawa Pria di Simalungun gegara Rebutan...,"[Hilang, Nyawa, Pria, di, Simalungun, gegara, ...",Hanya gegara rebutan mikrofon untuk bernyanyi ...,"[Hanya, gegara, rebutan, mikrofon, untuk, bern..."
4,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,"[Utang, Rp, 3, Juta, Bikin, Pegawai, Akper, Te...",Hidup pria bernama Monika Hutauruk (45) harus ...,"[Hidup, pria, bernama, Monika, Hutauruk, (, 45..."


### Casefolding


In [10]:
df['judul_tokens_lower'] = df['judul_tokens'].apply(lambda tokens: [token.lower() for token in tokens])
df['isi_berita_tokens_lower'] = df['isi_berita_tokens'].apply(lambda tokens: [token.lower() for token in tokens])

df[['judul', 'judul_tokens_lower', 'isi_berita_tokens_lower']].head()

,judul,judul_tokens_lower,isi_berita_tokens_lower
0,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,"[4, anak, di, bawah, umur, tersangka, pembunuh...","[pelaku, pembunuhan, dan, pemerkosaan, aa, (, ..."
1,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,"[26, tersangka, dibekuk, selama, 3, bulan, ter...","[dalam, waktu, kurang, lebih, 3, bulan, ,, pol..."
2,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,"[pekerja, kafe, disiram, air, keras, hingga, w...","[seorang, pekerja, kafe, di, cengkareng, ,, ma..."
3,Hilang Nyawa Pria di Simalungun gegara Rebutan...,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebutan, mikrofon, untuk, bern..."
4,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, bernama, monika, hutauruk, (, 45..."


### Stemming

In [12]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['judul_stemmed'] = df['judul_tokens_lower'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
df['isi_stemmed'] = df['isi_berita_tokens_lower'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

df[['judul', 'judul_stemmed', 'isi_stemmed']].head()

,judul,judul_stemmed,isi_stemmed
0,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,"[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, , 14, , orang,..."
1,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,"[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, , polr..."
2,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,"[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, , mas, , ..."
3,Hilang Nyawa Pria di Simalungun gegara Rebutan...,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi..."
4,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, , 45, , ..."


### Stop Word Removal & POS Tagging

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
stopword_factory = StopWordRemoverFactory()
stopwords = set(stopword_factory.get_stop_words())

def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords and token.strip() != '']

def pos_tag_tokens(tokens):
    return [(token, 'NN') for token in tokens]

# Stop word removal
df['judul_stemmed_nostop'] = df['judul_stemmed'].apply(remove_stopwords)
df['isi_stemmed_nostop'] = df['isi_stemmed'].apply(remove_stopwords)

# POS Tagging
df['judul_pos_nostop'] = df['judul_stemmed_nostop'].apply(pos_tag_tokens)
df['isi_pos_nostop'] = df['isi_stemmed_nostop'].apply(pos_tag_tokens)

df[['judul', 'judul_stemmed_nostop', 'judul_pos_nostop', 'isi_stemmed_nostop', 'isi_pos_nostop']].head()

,judul,judul_stemmed_nostop,judul_pos_nostop,isi_stemmed_nostop,isi_pos_nostop
0,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,"[4, anak, bawah, umur, sangka, bunuh, palemban...","[(4, NN), (anak, NN), (bawah, NN), (umur, NN),...","[laku, bunuh, perkosa, aa, 14, orang, remaja, ...","[(laku, NN), (bunuh, NN), (perkosa, NN), (aa, ..."
1,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,"[26, sangka, bekuk, lama, 3, bulan, akhir, kot...","[(26, NN), (sangka, NN), (bekuk, NN), (lama, N...","[waktu, kurang, lebih, 3, bulan, polres, probo...","[(waktu, NN), (kurang, NN), (lebih, NN), (3, N..."
2,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,"[kerja, kafe, siram, air, keras, hingga, wajah...","[(kerja, NN), (kafe, NN), (siram, NN), (air, N...","[orang, kerja, kafe, cengkareng, mas, 32, duga...","[(orang, NN), (kerja, NN), (kafe, NN), (cengka..."
3,Hilang Nyawa Pria di Simalungun gegara Rebutan...,"[hilang, nyawa, pria, simalungun, gegara, rebu...","[(hilang, NN), (nyawa, NN), (pria, NN), (simal...","[gegara, rebut, mikrofon, nyanyi, warung, tuak...","[(gegara, NN), (rebut, NN), (mikrofon, NN), (n..."
4,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[(utang, NN), (rp, NN), (3, NN), (juta, NN), (...","[hidup, pria, nama, monika, hutauruk, 45, akhi...","[(hidup, NN), (pria, NN), (nama, NN), (monika,..."


In [19]:
df[['judul', 'judul_stemmed_nostop', 'judul_pos_nostop', 'isi_stemmed_nostop', 'isi_pos_nostop']].head()

,judul,judul_stemmed_nostop,judul_pos_nostop,isi_stemmed_nostop,isi_pos_nostop
0,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,"[4, anak, bawah, umur, sangka, bunuh, palemban...","[(4, NN), (anak, NN), (bawah, NN), (umur, NN),...","[laku, bunuh, perkosa, aa, 14, orang, remaja, ...","[(laku, NN), (bunuh, NN), (perkosa, NN), (aa, ..."
1,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,"[26, sangka, bekuk, lama, 3, bulan, akhir, kot...","[(26, NN), (sangka, NN), (bekuk, NN), (lama, N...","[waktu, kurang, lebih, 3, bulan, polres, probo...","[(waktu, NN), (kurang, NN), (lebih, NN), (3, N..."
2,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,"[kerja, kafe, siram, air, keras, hingga, wajah...","[(kerja, NN), (kafe, NN), (siram, NN), (air, N...","[orang, kerja, kafe, cengkareng, mas, 32, duga...","[(orang, NN), (kerja, NN), (kafe, NN), (cengka..."
3,Hilang Nyawa Pria di Simalungun gegara Rebutan...,"[hilang, nyawa, pria, simalungun, gegara, rebu...","[(hilang, NN), (nyawa, NN), (pria, NN), (simal...","[gegara, rebut, mikrofon, nyanyi, warung, tuak...","[(gegara, NN), (rebut, NN), (mikrofon, NN), (n..."
4,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[(utang, NN), (rp, NN), (3, NN), (juta, NN), (...","[hidup, pria, nama, monika, hutauruk, 45, akhi...","[(hidup, NN), (pria, NN), (nama, NN), (monika,..."


In [20]:
df.to_csv("../data/processed/prepocessed_all_data.csv", sep=',', index=False)